## NLP pre-processing
Before embarking on any NLP techniques (i.e. Sentiment Analysis, etc.) we need to make sure the text data is in the proper format.
If not, the text won't be accepted into any models or processes.

First we must import some libraries
spaCy will be the main library we use for all NLP actions.  spaCy is efficient, and easy to use.  
NLTK has some useful features as well, but isn't built for quick, simple nlp.

In [ ]:
import pandas as pd
import spacy
import nltk
from nltk import word_tokenize
nltk.download()
import en_core_web_sm  # or any other model you downloaded via spacy download or pip
nlp = en_core_web_sm.load()

Next you'll need to build your dataframe
This is easily done using the "Connect to SQL Server" and "Use SQL" code snippets replicated below and available here: \\es00cifs00\users$\egagne\WINNT\System\Desktop\Python Code Snippets

For this example we'll use the APPRTIP table

In [ ]:
# assign server and db
server = 'ES11vADOSQL006'
db = 'master'

# Create the connection to all dbs
cnxn = pyodbc.connect('DRIVER={ODBC Driver 11 for SQL Server};SERVER=ES11vADOSQL006;DATABASE=master;Trusted_Connection=yes;')

In [ ]:
# Pull data from APPRTIP
#Create an additional column will all text concatenated
#Filter on IsSubmitted = 'Y' and TIPEndedAppeal = 'N'
sql3 = """

SELECT *
,(TIPImprovementPlan1 + ' ' + TIPActionPlan + ' ' + TIPTimeLinePlan + ' ' + TIPSupportPlan + ' ' + TIPAssessmentPlan) as TIP_all_txt
FROM [APPR_EXT].[dbo].[APPRTIP]
where IsSubmitted = 'Y' and TIPEndedAppeal = 'N'

"""
TIP_with_rats = pd.io.sql.read_sql(sql3, cnxn)
TIP_with_rats.head()

It's easiest to process the text if it's not in the dataframe.  However, ultimately we will appreciate having the dataframe structure.  So let's pull the text out of the dataframe, do some processing, and then stick it back in

In [ ]:
#First, let's create 3 empty lists.  This is where we'll put the processed data for holding until we merge it back in with the dataframe.

tokens = []
lemma = []
pos = []

#Next we push our text through the nlp pipe

for doc in nlp.pipe(TIP_with_rats['TIPImprovementPlan1'].astype('unicode').values, batch_size=9845,
                        n_threads=3):
    
#Here we're filling in our empty lists with the text, if it meets our set conditions
#Basically we're saying if the word is not punctution and not a stop word and not extra whitespace then 

    if doc.is_parsed:
        tokens.append([n.text for n in doc if not n.is_punct and not n.is_stop and not n.is_space])
        lemma.append([n.lemma_ for n in doc if not n.is_punct and not n.is_stop and not n.is_space])
        pos.append([n.pos_ for n in doc if not n.is_punct and not n.is_stop and not n.is_space])
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

#Now we create new columns in our datafram and populate them with the lists        
TIP_with_rats['s_tokens_IP'] = tokens
TIP_with_rats['s_lemmas_IP'] = lemma 
TIP_with_rats['s_pos_IP'] = pos

TIP_with_rats.head()